In [4]:
import numpy as np
import matplotlib.pylab as plt
import scipy.io
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
from tensorflow import keras
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import absl.logging
import tensorflow as tf


def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

absl.logging.set_verbosity(absl.logging.ERROR) # .h5 에러메세지 무시

mat_file = scipy.io.loadmat("data/PTSDvsHC.mat")

dataset = mat_file['dataset'] 
# 90개의 데이터, 62개의 전극 채널, 1001의 타임스텝 - 1001씩 이미 에포킹을 한 상태이며, 그에따른 평균을 낸 것임(에포킹에 대해)

subject_test_acc = []

X = dataset
y = mat_file['Y']
y = to_categorical(y, num_classes = 2)

X = X.reshape(90, 1001, 62, 1)

input_shape = (1001, 62, 1)

for i in range(len(dataset)): # loso
    X_train = np.concatenate([X[:i], X[i+1:]], axis=0)
    X_test = X[i:i+1]
    y_train = np.concatenate([y[:i], y[i+1:]], axis=0)
    y_test = y[i:i+1]

    # print(f"X_train_shape : {X_train.shape}")
    # print(f"X_test_shape : {X_test.shape}")
    # print(f"y_train_shape : {y_train.shape}")
    # print(f"y_test_shape : {y_test.shape}")
    
    model_input = layers.Input(shape = input_shape)
    br1 = layers.Conv2D(32, (40, 1), padding = 'same')(model_input) # temporal filter
    br1 = layers.Activation(mish)(br1) # result1_matrix, shape=(1001, 62, 32)
    # print(br1.shape)

    input_shape_tr = layers.Permute((2, 1, 3))(model_input) # (62, 1001, 1)
    br2 = layers.Conv2D(32, (62, 1), padding = 'same')(input_shape_tr)
    br2 = layers.Activation(mish)(br2) # result2_matrix, shape=(62, 1001, 32)

    # br1, br2 matrix multiply
    br1 = layers.Permute((3, 1, 2))(br1) # (32, 1001, 62)
    br2 = layers.Permute((3, 1, 2))(br2) # (32, 62, 1001)
    br_mul = layers.Lambda(lambda x: tf.matmul(x[0], x[1]))([br2, br1])
    # print(br_mul.shape)  # (32, 62, 62)

    # branch matrix multiply convolution
    br_mul = layers.Permute((2, 3, 1))(br_mul) # (62, 62, 32) <- 풀링하기 위함
    print(br_mul)
    br_mul = layers.MaxPooling2D(pool_size=(62, 62))(br_mul) # (62, 62)
    br_mul = layers.Conv2D(32, (3, 3), padding = 'same')(br_mul)
    br_mul = layers.Activation(mish)(br_mul)

    end = layers.Flatten()(br_mul)

    # globalmaxpooling을 할 경우, (batch_size, channel)로 output이 나오기 때문에 사용하지 않기로 함
    # maxpooling 사용

    prediction = layers.Dense(2, activation = 'softmax')(end)
    model = keras.Model(inputs = model_input, outputs = prediction)
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    history = model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=1, batch_size=1, verbose=0)
    predictions = model.predict(X_test)
    
    predicted_classes = np.argmax(predictions, axis=1)
    actual_classes = np.argmax(y_test, axis=1)

    acc = accuracy_score(predicted_classes, actual_classes)

    # model.summary()
    
    print(f"fold {i} accuracy : {acc}")

    subject_test_acc.append(acc)
print(f"loso mean accuracy : {np.mean(subject_test_acc)}")

<KerasTensor shape=(None, 62, 62, 32), dtype=float32, sparse=False, name=keras_tensor_192>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
fold 0 accuracy : 1.0
<KerasTensor shape=(None, 62, 62, 32), dtype=float32, sparse=False, name=keras_tensor_210>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
fold 1 accuracy : 1.0
<KerasTensor shape=(None, 62, 62, 32), dtype=float32, sparse=False, name=keras_tensor_228>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
fold 2 accuracy : 1.0
<KerasTensor shape=(None, 62, 62, 32), dtype=float32, sparse=False, name=keras_tensor_246>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
fold 3 accuracy : 0.0
<KerasTensor shape=(None, 62, 62, 32), dtype=float32, sparse=False, name=keras_tensor_264>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
fold 4 accuracy : 0.0
<KerasTensor shape=(None, 62, 62, 32), dtype=float32, sparse=False, name=keras_tensor_282>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
fold 5 accuracy : 0.0
<KerasTensor shape=(None, 62, 62, 32), dtype=float32, sparse=False, name=keras_tensor_30

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import scipy.io
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
from tensorflow import keras
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import absl.logging
import tensorflow as tf


def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

absl.logging.set_verbosity(absl.logging.ERROR) # .h5 에러메세지 무시

mat_file = scipy.io.loadmat("data/PTSDvsHC.mat")

dataset = mat_file['dataset'] 
# 90개의 데이터, 62개의 전극 채널, 1001의 타임스텝 - 1001씩 이미 에포킹을 한 상태이며, 그에따른 평균을 낸 것임(에포킹에 대해)

subject_test_acc = []

X = dataset
y = mat_file['Y']
y = to_categorical(y, num_classes = 2)

X = X.reshape(90, 1001, 62, 1)

input_shape = (1001, 62, 1)

for i in range(len(dataset)): # loso
    X_train = np.concatenate([X[:i], X[i+1:]], axis=0)
    X_test = X[i:i+1]
    y_train = np.concatenate([y[:i], y[i+1:]], axis=0)
    y_test = y[i:i+1]

    # print(f"X_train_shape : {X_train.shape}")
    # print(f"X_test_shape : {X_test.shape}")
    # print(f"y_train_shape : {y_train.shape}")
    # print(f"y_test_shape : {y_test.shape}")
    
    model_input = layers.Input(shape = input_shape)
    br1 = layers.Conv2D(32, (40, 1), padding = 'same')(model_input) # temporal filter
    br1 = layers.Activation(mish)(br1) # result1_matrix, shape=(1001, 62, 32)
    # print(br1.shape)

    input_shape_tr = layers.Permute((2, 1, 3))(model_input) # (62, 1001, 1)
    br2 = layers.Conv2D(32, (62, 1), padding = 'same')(input_shape_tr)
    br2 = layers.Activation(mish)(br2) # result2_matrix, shape=(62, 1001, 32)

    # br1, br2 matrix multiply
    br1 = layers.Permute((3, 1, 2))(br1) # (32, 1001, 62)
    br2 = layers.Permute((3, 1, 2))(br2) # (32, 62, 1001)
    br_mul = layers.Lambda(lambda x: tf.matmul(x[0], x[1]))([br2, br1])
    # print(br_mul.shape)  # (32, 62, 62)

    # branch matrix multiply convolution
    br_mul = layers.Permute((2, 3, 1))(br_mul) # (62, 62, 32) <- 풀링하기 위함
    print(br_mul)
    br_mul = layers.MaxPooling2D(pool_size=(62, 62))(br_mul) # (62, 62)
    br_mul = layers.Conv2D(32, (3, 3), padding = 'same')(br_mul)
    br_mul = layers.Activation(mish)(br_mul)

    end = layers.Flatten()(br_mul)

    # globalmaxpooling을 할 경우, (batch_size, channel)로 output이 나오기 때문에 사용하지 않기로 함
    # maxpooling 사용

    prediction = layers.Dense(2, activation = 'softmax')(end)
    model = keras.Model(inputs = model_input, outputs = prediction)
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    history = model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=200, batch_size=1, verbose=2)
    predictions = model.predict(X_test)
    
    predicted_classes = np.argmax(predictions, axis=1)
    actual_classes = np.argmax(y_test, axis=1)

    acc = accuracy_score(predicted_classes, actual_classes)

    # model.summary()
    
    print(f"fold {i} accuracy : {acc}")

    subject_test_acc.append(acc)
print(f"loso mean accuracy : {np.mean(subject_test_acc)}")

<KerasTensor shape=(None, 62, 62, 32), dtype=float32, sparse=False, name=keras_tensor_1812>
Epoch 1/200
89/89 - 4s - 48ms/step - accuracy: 0.5169 - loss: 12.4542 - val_accuracy: 0.0000e+00 - val_loss: 7.6014
Epoch 2/200
89/89 - 3s - 35ms/step - accuracy: 0.4944 - loss: 6.4179 - val_accuracy: 1.0000 - val_loss: 0.0358
Epoch 3/200
89/89 - 3s - 36ms/step - accuracy: 0.6180 - loss: 2.1117 - val_accuracy: 1.0000 - val_loss: 0.6100
Epoch 4/200
89/89 - 3s - 36ms/step - accuracy: 0.7528 - loss: 0.8440 - val_accuracy: 0.0000e+00 - val_loss: 2.2265
Epoch 5/200
89/89 - 3s - 36ms/step - accuracy: 0.7191 - loss: 1.1166 - val_accuracy: 0.0000e+00 - val_loss: 1.5665
Epoch 6/200
89/89 - 3s - 37ms/step - accuracy: 0.6966 - loss: 1.7202 - val_accuracy: 1.0000 - val_loss: 5.4083e-04
Epoch 7/200
89/89 - 3s - 35ms/step - accuracy: 0.5056 - loss: 13.6208 - val_accuracy: 1.0000 - val_loss: 0.2337
Epoch 8/200
89/89 - 3s - 34ms/step - accuracy: 0.5618 - loss: 3.0325 - val_accuracy: 0.0000e+00 - val_loss: 4.320